# Demonstrate Seg-Grad-CAM on a U-Net with a backbone from 'segmentation_models' trained on Cityscapes

## 0. Imports 

In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from pathlib import Path
import sys
import matplotlib.pyplot as plt

import segmentation_models as sm

from seggradcam.dataloaders import Cityscapes
#from metrics import iou_coef, dice_coef, dice_loss
from seggradcam.unet import csbd_unet, manual_unet, TrainUnet
from seggradcam.training_write import TrainingParameters, TrainingResults
from seggradcam.training_plots import plot_predict_and_gt, plot_loss, plot_metric
from seggradcam.seggradcam import SegGradCAM, SuperRoI, ClassRoI, PixelRoI, BiasRoI
from seggradcam.visualize_sgc import SegGradCAMplot

Using TensorFlow backend.


Segmentation Models: using `keras` framework.


## 1. Dataset set up

In [2]:
BATCH_SIZE = 4
trainparam = TrainingParameters(
    epochs = 50,
    scale = 4,
    batch_size = BATCH_SIZE, 
    n_train= 2975, #2975 max
    n_val = 500 # 500 max 
    ,steps_per_epoch = int(3000/BATCH_SIZE)
    ,validation_steps = int(500/BATCH_SIZE)   
    )
trainparam.saveToJson()

trainset = Cityscapes(n = trainparam.n_train, shuffle = True, scale = trainparam.scale, prefix = "train",normalize=True)
valset = Cityscapes(n = trainparam.n_val, shuffle = False, scale = trainparam.scale, prefix = "val",normalize=True)

## 1.1 A. Prepare dataset

In [6]:
path = "C:\\Users\\vinograd\\Documents\\cityscapes\\leftImg8bit_trainvaltest\\leftImg8bit"
#path = "../../inputs/cityscapes/leftImg8bit_trainvaltest/leftImg8bit"
trainset.get_and_save_npz(path)
valset.get_and_save_npz(path)

Root path:  C:\Users\vinograd\Documents\cityscapes\leftImg8bit_trainvaltest\leftImg8bit\train
Length of the set: 2975 .  2975  will be loaded.


100%|██████████████████████████████████████████████████████████████████████████████| 2975/2975 [01:43<00:00, 28.61it/s]


Root path:  C:\Users\vinograd\Documents\cityscapes\leftImg8bit_trainvaltest\leftImg8bit\val
Length of the set: 500 .  500  will be loaded.


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:18<00:00, 27.35it/s]


## 1.1 B. Load prepared dataset

In [3]:
trainset.load_npz()
valset.load_npz()

## 2. U-Net set up

In [5]:
warnings.filterwarnings('ignore')
BACKBONE = 'vgg16'
preprocess_input = sm.get_preprocessing(BACKBONE)
n_classes = trainparam.n_classes
activation = 'sigmoid' if n_classes == 1 else 'softmax'
#create model
model = sm.Unet(BACKBONE, classes=n_classes, activation=activation, encoder_weights='imagenet')

## 2.1 A. Train a U-Net yourself 

In [9]:
LR = 0.00045
EPOCHS = 2 # We recommend training for at least 150
"""
# check shapes for errors
assert train_dataloader[0][0].shape == (BATCH_SIZE, 320, 320, 3)
assert train_dataloader[0][1].shape == (BATCH_SIZE, 320, 320, n_classes)
"""
# define callbacks for learning rate scheduling and best checkpoints saving
import keras
from datetime import datetime
timestamp = datetime.now()
mdir = '../../output/sm_'+BACKBONE+'_'+str(timestamp.strftime("%m_%d_%H_%M"))
try:
    os.mkdir(mdir)
except OSError as error: 
    print(error) 
callbacks = [
    keras.callbacks.ModelCheckpoint(mdir+'best_model.h5', save_weights_only=True, save_best_only=True, mode='min'),
    keras.callbacks.ReduceLROnPlateau(),
]

[WinError 183] Cannot create a file when that file already exists: '../../output/sm_vgg16_09_30_16_14'


In [10]:
optim = keras.optimizers.Adam(LR)

dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.BinaryFocalLoss() if n_classes == 1 else sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

model.compile(optim, total_loss, metrics)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
            rescale=1. / 255,
            # brightness_range = (.8,1.2),
            horizontal_flip=True)
train_input = datagen.flow(trainset.X, trainset.Y, batch_size=BATCH_SIZE)
val_input = datagen.flow(valset.X, valset.Y, batch_size=BATCH_SIZE) 

history = model.fit_generator(train_input,
                                      validation_data=val_input,
                                      epochs=EPOCHS,
                                      steps_per_epoch=trainparam.steps_per_epoch,
                                      validation_steps=trainparam.validation_steps,
                                      callbacks=callbacks)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/2


120/750 [===>..........................] - ETA: 4:49:14 - loss: 0.9501 - iou_score: 0.0131 - f1-score: 0.024 - ETA: 3:42:37 - loss: 0.9367 - iou_score: 0.0190 - f1-score: 0.033 - ETA: 3:23:34 - loss: 0.9280 - iou_score: 0.0277 - f1-score: 0.045 - ETA: 3:11:27 - loss: 0.9200 - iou_score: 0.0297 - f1-score: 0.048 - ETA: 3:04:52 - loss: 0.9111 - iou_score: 0.0323 - f1-score: 0.052 - ETA: 2:51:58 - loss: 0.9051 - iou_score: 0.0328 - f1-score: 0.053 - ETA: 2:41:27 - loss: 0.8976 - iou_score: 0.0370 - f1-score: 0.058 - ETA: 2:33:23 - loss: 0.8932 - iou_score: 0.0372 - f1-score: 0.058 - ETA: 2:27:16 - loss: 0.8874 - iou_score: 0.0395 - f1-score: 0.061 - ETA: 2:22:44 - loss: 0.8813 - iou_score: 0.0420 - f1-score: 0.063 - ETA: 2:19:30 - loss: 0.8760 - iou_score: 0.0441 - f1-score: 0.065 - ETA: 2:16:32 - loss: 0.8706 - iou_score: 0.0453 - f1-score: 0.066 - ETA: 2:13:45 - loss: 0.8650 - iou_score: 0.0470 - f1-score: 0.068 - ETA: 2:11:16 - loss: 0.8613 - iou_score: 0.0482 - f1-score: 0.069 - ETA: 

240/750 [========>.....................] - ETA: 1:24:37 - loss: 0.7061 - iou_score: 0.2246 - f1-score: 0.300 - ETA: 1:24:25 - loss: 0.7050 - iou_score: 0.2259 - f1-score: 0.301 - ETA: 1:24:14 - loss: 0.7043 - iou_score: 0.2266 - f1-score: 0.302 - ETA: 1:24:02 - loss: 0.7041 - iou_score: 0.2268 - f1-score: 0.303 - ETA: 1:23:50 - loss: 0.7035 - iou_score: 0.2275 - f1-score: 0.304 - ETA: 1:23:40 - loss: 0.7030 - iou_score: 0.2281 - f1-score: 0.304 - ETA: 1:23:28 - loss: 0.7020 - iou_score: 0.2293 - f1-score: 0.306 - ETA: 1:23:16 - loss: 0.7014 - iou_score: 0.2300 - f1-score: 0.307 - ETA: 1:23:08 - loss: 0.7013 - iou_score: 0.2301 - f1-score: 0.307 - ETA: 1:23:00 - loss: 0.7009 - iou_score: 0.2305 - f1-score: 0.307 - ETA: 1:22:50 - loss: 0.6998 - iou_score: 0.2316 - f1-score: 0.309 - ETA: 1:22:40 - loss: 0.6990 - iou_score: 0.2323 - f1-score: 0.310 - ETA: 1:22:31 - loss: 0.6981 - iou_score: 0.2333 - f1-score: 0.311 - ETA: 1:22:21 - loss: 0.6979 - iou_score: 0.2335 - f1-score: 0.311 - ETA: 

362/750 [=============>................] - ETA: 1:06:26 - loss: 0.6588 - iou_score: 0.2729 - f1-score: 0.363 - ETA: 1:06:18 - loss: 0.6584 - iou_score: 0.2733 - f1-score: 0.363 - ETA: 1:06:10 - loss: 0.6581 - iou_score: 0.2736 - f1-score: 0.364 - ETA: 1:06:01 - loss: 0.6580 - iou_score: 0.2737 - f1-score: 0.364 - ETA: 1:05:53 - loss: 0.6578 - iou_score: 0.2738 - f1-score: 0.364 - ETA: 1:05:45 - loss: 0.6575 - iou_score: 0.2741 - f1-score: 0.364 - ETA: 1:05:36 - loss: 0.6573 - iou_score: 0.2743 - f1-score: 0.365 - ETA: 1:05:28 - loss: 0.6576 - iou_score: 0.2741 - f1-score: 0.364 - ETA: 1:05:20 - loss: 0.6573 - iou_score: 0.2743 - f1-score: 0.365 - ETA: 1:05:11 - loss: 0.6573 - iou_score: 0.2743 - f1-score: 0.365 - ETA: 1:05:03 - loss: 0.6571 - iou_score: 0.2744 - f1-score: 0.365 - ETA: 1:04:54 - loss: 0.6566 - iou_score: 0.2749 - f1-score: 0.366 - ETA: 1:04:46 - loss: 0.6562 - iou_score: 0.2753 - f1-score: 0.366 - ETA: 1:04:38 - loss: 0.6560 - iou_score: 0.2754 - f1-score: 0.366 - ETA: 

486/750 [==================>...........] - ETA: 50:09 - loss: 0.6363 - iou_score: 0.2944 - f1-score: 0.391 - ETA: 50:01 - loss: 0.6362 - iou_score: 0.2946 - f1-score: 0.391 - ETA: 49:53 - loss: 0.6362 - iou_score: 0.2946 - f1-score: 0.391 - ETA: 49:45 - loss: 0.6361 - iou_score: 0.2946 - f1-score: 0.391 - ETA: 49:37 - loss: 0.6360 - iou_score: 0.2947 - f1-score: 0.391 - ETA: 49:29 - loss: 0.6359 - iou_score: 0.2948 - f1-score: 0.391 - ETA: 49:21 - loss: 0.6358 - iou_score: 0.2949 - f1-score: 0.391 - ETA: 49:13 - loss: 0.6357 - iou_score: 0.2950 - f1-score: 0.392 - ETA: 49:05 - loss: 0.6355 - iou_score: 0.2951 - f1-score: 0.392 - ETA: 48:57 - loss: 0.6354 - iou_score: 0.2953 - f1-score: 0.392 - ETA: 48:49 - loss: 0.6353 - iou_score: 0.2954 - f1-score: 0.392 - ETA: 48:41 - loss: 0.6350 - iou_score: 0.2956 - f1-score: 0.392 - ETA: 48:33 - loss: 0.6349 - iou_score: 0.2957 - f1-score: 0.392 - ETA: 48:25 - loss: 0.6350 - iou_score: 0.2956 - f1-score: 0.392 - ETA: 48:17 - loss: 0.6351 - iou_s

610/750 [=======================>......] - ETA: 33:52 - loss: 0.6251 - iou_score: 0.3055 - f1-score: 0.405 - ETA: 33:45 - loss: 0.6250 - iou_score: 0.3055 - f1-score: 0.405 - ETA: 33:37 - loss: 0.6249 - iou_score: 0.3056 - f1-score: 0.406 - ETA: 33:29 - loss: 0.6250 - iou_score: 0.3055 - f1-score: 0.406 - ETA: 33:21 - loss: 0.6249 - iou_score: 0.3056 - f1-score: 0.406 - ETA: 33:13 - loss: 0.6247 - iou_score: 0.3058 - f1-score: 0.406 - ETA: 33:05 - loss: 0.6248 - iou_score: 0.3058 - f1-score: 0.406 - ETA: 32:58 - loss: 0.6247 - iou_score: 0.3059 - f1-score: 0.406 - ETA: 32:50 - loss: 0.6247 - iou_score: 0.3059 - f1-score: 0.406 - ETA: 32:42 - loss: 0.6245 - iou_score: 0.3061 - f1-score: 0.406 - ETA: 32:34 - loss: 0.6245 - iou_score: 0.3060 - f1-score: 0.406 - ETA: 32:26 - loss: 0.6246 - iou_score: 0.3060 - f1-score: 0.406 - ETA: 32:18 - loss: 0.6246 - iou_score: 0.3061 - f1-score: 0.406 - ETA: 32:11 - loss: 0.6246 - iou_score: 0.3060 - f1-score: 0.406 - ETA: 32:03 - loss: 0.6246 - iou_s

622/750 [=======================>......] - ETA: 17:49 - loss: 0.6191 - iou_score: 0.3116 - f1-score: 0.413 - ETA: 17:41 - loss: 0.6190 - iou_score: 0.3117 - f1-score: 0.414 - ETA: 17:33 - loss: 0.6189 - iou_score: 0.3118 - f1-score: 0.414 - ETA: 17:26 - loss: 0.6189 - iou_score: 0.3117 - f1-score: 0.414 - ETA: 17:18 - loss: 0.6189 - iou_score: 0.3118 - f1-score: 0.414 - ETA: 17:10 - loss: 0.6188 - iou_score: 0.3118 - f1-score: 0.414 - ETA: 17:03 - loss: 0.6188 - iou_score: 0.3119 - f1-score: 0.414 - ETA: 16:55 - loss: 0.6186 - iou_score: 0.3120 - f1-score: 0.414 - ETA: 16:47 - loss: 0.6186 - iou_score: 0.3120 - f1-score: 0.414 - ETA: 16:39 - loss: 0.6186 - iou_score: 0.3121 - f1-score: 0.414 - ETA: 16:32 - loss: 0.6185 - iou_score: 0.3121 - f1-score: 0.414 - ETA: 16:24 - loss: 0.6184 - iou_score: 0.3122 - f1-score: 0.4147

### Training plots

In [ ]:
# Plot training & validation iou_score values
plt.figure(figsize=(30, 5))
plt.subplot(121)
plt.plot(history.history['iou_score'])
plt.plot(history.history['val_iou_score'])
plt.plot(history.history['f1-score'])
plt.plot(history.history['val_f1-score'])
plt.title('Model IoU & F1 scores')
plt.ylabel('Score')
plt.xlabel('Epoch')
plt.legend(['Train IoU', 'Val IoU','Train F1','Val F1'], loc='upper left')

# Plot training & validation loss values
plt.subplot(122)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.savefig(mdir+'iou_f1_loss.png')

## 2.1 B. Load a U-Net 

In [ ]:
mdir = '../../output/unet_backbone_16_07/'
model.load_weights(mdir+'best_model.h5')

## 2.2. Take a look at predictions

In [ ]:
plot_predict_and_gt(model, valset.X, valset.Y, range(0,10,1), mdir, n_classes)

# 3. Seg-Grad-CAM

## 3.1 Set up layers for propagation of gradients

In [ ]:
prop_from_layer = model.layers[-1].name
prop_to_layer = 'center_block1_relu'
# We recommend to try also:
#prop_to_layer = 'center_block2_relu'

## 3.2 Choose an image

In [ ]:
cls = 7 # car
imid = 3
image =  valset.X[imid]

## 3.3 A. Seg-Grad-CAM for a pixel

In [ ]:
# create a SegGradCAM object
roi=PixelRoI(500,600,image)
pixsgc = SegGradCAM(trainunet.model, image, cls,  prop_to_layer,prop_from_layer, roi=roi,
                 normalize=True, abs_w=False, posit_w=False)
# compute SegGradCAM
pixsgc.SGC()
# create an object with plotting functionality
plotter = SegGradCAMplot(pixsgc,trainunet, gt = trainset.Y[imid])
# plot explanations on 1 picture
plotter.explainPixel()

In [ ]:
#Plot 4 images: original, ground truth, predicted mask, seg-grad-cam explanations for a selected single pixel
plotter.pixelGtPrediction()

## 3.3 B. Seg-Grad-CAM for the class (all pixels that were predicted as class 'cls')

In [ ]:
clsroi = ClassRoI(model=model,image=image,cls=cls)
newsgc = SegGradCAM(model, image, cls, prop_to_layer,  prop_from_layer, roi=clsroi,
                 normalize=True, abs_w=False, posit_w=False)
newsgc.SGC()

# create an object with plotting functionality
plotter = SegGradCAMplot(newsgc,model=model,n_classes=n_classes,outfolder=mdir, gt = valset.Y[imid])
# plot explanations on 1 picture
plotter.explainClass()

In [ ]:
plotter.classGtPrediction()

## 3.3 C. Seg-Grad-CAM for a region of interest
### 3.3.C.1 for the largest set of connected pixels predicted as 'cls'

In [ ]:
clsroi.largestComponent()
newsgc = SegGradCAM(model, image, cls, prop_to_layer,  prop_from_layer, roi=clsroi,
                 normalize=True, abs_w=False, posit_w=False)
newsgc.SGC()

plotter = SegGradCAMplot(newsgc,model=model,n_classes=n_classes,outfolder=mdir, gt = valset.Y[imid])
plotter.explainRoi()

In [ ]:
plotter.roiGtPrediction()

### 3.3.C.2 for the smallest set of connected pixels predicted as 'cls'

In [ ]:
clsroi.smallestComponent()
newsgc = SegGradCAM(model, image, cls, prop_to_layer,  prop_from_layer, roi=clsroi,
                 normalize=True, abs_w=False, posit_w=False)
newsgc.SGC()

plotter = SegGradCAMplot(newsgc,model=model,n_classes=n_classes,outfolder=mdir, gt = valset.Y[imid])
plotter.explainRoi()

In [ ]:
plotter.roiGtPrediction()

### 3.3.C.3 It is also possible to define an arbitrary RoI